# Tarea 2: Redes Convolucionales
Integrantes:
- Kurt Castro
- Diego León
- Cristián Pizarro

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras import models, layers, optimizers, regularizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import random
import kagglehub
# Establecer la semilla para Python
random.seed(42)

# Establecer la semilla para NumPy
np.random.seed(42)

# Establecer la semilla para TensorFlow
tf.random.set_seed(42)

Activar GPU en "Entorno de Ejecución". No lo olviden o estarán horas entrenando!

In [ ]:
# Verifica las GPUs disponibles
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Se encontró GPU: {gpus}")
else:
    print("No se encontró GPU. Asegúrate de haber activado el entorno con GPU en Colab.")

Se encontró GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Download latest version
path = kagglehub.dataset_download("samuelcortinhas/muffin-vs-chihuahua-image-classification")

print("Path to dataset files:", path)

100%|██████████| 474M/474M [00:21<00:00, 22.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification/versions/2


In [ ]:
base_path = "/root/.cache/kagglehub/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification/versions/2"

In [ ]:
#image generator
data_generator = ImageDataGenerator(rescale=1./255,
                                    vertical_flip=True,
                                    horizontal_flip=True)

# train y test
train_dir = f'{base_path}/train'
test_dir = f'{base_path}/test'

batch_size = 16
num_classes = 2


# generators
image_size = (150, 150)
input_shape = (150, 150, 3)

# Cargar datos directamente
X_train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=image_size,
    batch_size=batch_size
)

X_test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='binary',
    image_size=image_size,
    batch_size=batch_size
)

# Convertir a tensores numpy si es necesario
X_train, y_train = [], []
for images, labels in X_train_ds:
    X_train.append(images.numpy())
    y_train.append(labels.numpy())

X_test, y_test = [], []
for images, labels in X_test_ds:
    X_test.append(images.numpy())
    y_test.append(labels.numpy())

# Combinar los datos en arreglos numpy
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train).ravel()
X_test = np.concatenate(X_test)
y_test = np.concatenate(y_test).ravel()

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.
X_train shape: (4733, 150, 150, 3), y_train shape: (4733,)
X_test shape: (1184, 150, 150, 3), y_test shape: (1184,)


## MLP

In [ ]:
# Dimensiones originales de las imágenes
num_samples, height, width, channels = X_train.shape

# Aplanar las imágenes
X_train_flattened = X_train.reshape(num_samples, height * width * channels)

# Si tienes un conjunto de prueba
X_test_flattened = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2] * X_test.shape[3])

In [ ]:
hidden_units = 128
model_mlp = Sequential([
    Dense(hidden_units, activation='relu'),
    Dense(1, activation='sigmoid')  # Una sola salida para clasificación binaria
])

model_mlp.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(learning_rate=1e-2),
              metrics=['accuracy'])

#traint the parameters
history = model_mlp.fit(X_train_flattened, y_train,
                    epochs=10,
                    batch_size=batch_size)

#evalute the accuracy
train_acc=model_mlp.evaluate(X_train_flattened, y_train, batch_size=batch_size)[1]
test_acc=model_mlp.evaluate(X_test_flattened, y_test, batch_size=batch_size)[1]
print(f'Training accuracy: {train_acc:.2f}')
print(f'Test accuracy: {test_acc:.2f}')

Epoch 1/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5811 - loss: 11867.1836
Epoch 2/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6767 - loss: 0.8397
Epoch 3/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7098 - loss: 0.6570
Epoch 4/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7124 - loss: 0.6338
Epoch 5/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6993 - loss: 0.6390
Epoch 6/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6562 - loss: 0.6515
Epoch 7/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7025 - loss: 0.6239
Epoch 8/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7047 - loss: 0.6282
Epoch 9/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6474 - loss: 0.6844
Epoch 10/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5407 - loss: 0.6897
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5733 - loss: 0.6732
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6014 - 

## Red convolucional

In [ ]:
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')  # Una sola salida para clasificación binaria
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model_cnn.compile(loss=keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adadelta(),
             metrics=['accuracy'])

In [ ]:
X_train.shape

(4733, 150, 150, 3)

In [ ]:
y_train.shape

(4733,)

In [ ]:
history = model_cnn.fit(
X_train, y_train,
batch_size=batch_size,
epochs=10,
verbose = 1,
validation_data=(X_test, y_test)
)

#evalute the accuracy
train_acc=model_cnn.evaluate(X_train, y_train, batch_size=batch_size)[1]
test_acc=model_cnn.evaluate(X_test, y_test, batch_size=batch_size)[1]
print(f'Training accuracy: {train_acc:.2f}')
print(f'Test accuracy: {test_acc:.2f}')


Epoch 1/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6852 - loss: 0.0000e+00 - val_accuracy: 0.7508 - val_loss: 0.0000e+00
Epoch 2/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.7098 - loss: 0.0000e+00 - val_accuracy: 0.7610 - val_loss: 0.0000e+00
Epoch 3/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.7145 - loss: 0.0000e+00 - val_accuracy: 0.7644 - val_loss: 0.0000e+00
Epoch 4/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.7230 - loss: 0.0000e+00 - val_accuracy: 0.7677 - val_loss: 0.0000e+00
Epoch 5/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.7318 - loss: 0.0000e+00 - val_accuracy: 0.7753 - val_loss: 0.0000e+00
Epoch 6/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.7427 - loss: 0.0000e+00 - val_accuracy: 0.7821 - val_loss: 0.0000e+00
Epoch 7/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.7528 - loss: 0.0000e+00 - val_accuracy: 0.7863 - val_loss: 0.0000e+00
Epoch 8/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 5

/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7892 - loss: 0.0000e+00
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8208 - loss: 0.0000e+00
Training accuracy: 0.80
Test accuracy: 0.81
